In [1]:
import numpy as np
import soundfile as sf
import math
from scipy import signal
import scipy
import random
import sofa
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import sofa

In [30]:
def inverseIR(pathC,pathA,receiver,emitter=0):

    left_mea=278

    right_mea=314

    emitter=0

    receiver=0
    
   

    HRTF = sofa.Database.open(pathC)
 
    leftIR = HRTF.Data.IR.get_values(indices={'M':left_mea,'R':receiver,'E':emitter})

    leftIR=leftIR.reshape(leftIR.size,)

    rightIR= HRTF.Data.IR.get_values(indices={'M':right_mea,'R':receiver,'E':emitter})

    rightIR=rightIR.reshape(rightIR.size,)
    
    #fourier transoform .
    leftFT = scipy.fft.fft(leftIR,axis=-1) 
    
    rightFT = scipy.fft.fft(rightIR,axis=-1) 

    reverseL=1/leftFT

    reverseR=1/rightFT

    #inverse transform to get the time domain transfer function
    left = scipy.fft.ifft(reverseL,axis=-1) 
    
    right = scipy.fft.ifft(reverseR,axis=-1) 
    return (left,right)

In [31]:

left_mea=278

right_mea=314

emitter=0

receiver=0
    
pathC = '/Users/eeeds/Desktop/yg/mit_kemar_normal_pinna.sofa'
    
pathA=f'E://yonggang/hpcfE/Eaudio0001firsthalf.wav'   

filename='/Users/eeeds/Desktop/yg/duolaoa.wav'

HRTF = sofa.Database.open(pathC)
 
leftIR = HRTF.Data.IR.get_values(indices={'M':left_mea,'R':receiver,'E':emitter})

leftIR=leftIR.reshape(leftIR.size,)

rightIR= HRTF.Data.IR.get_values(indices={'M':right_mea,'R':receiver,'E':emitter})

rightIR=rightIR.reshape(rightIR.size,)
    
    #fourier transoform .
leftFT = scipy.fft.fft(leftIR,axis=-1) 
    
rightFT = scipy.fft.fft(rightIR,axis=-1) 

reverseL=1/leftFT

reverseR=1/rightFT

    #inverse transform to get the time domain transfer function
left = scipy.fft.ifft(reverseL,axis=-1) 
    
right = scipy.fft.ifft(reverseR,axis=-1) 

#print(right)
    #convolution
d, sr1 = sf.read(pathA)
    
left= left.reshape(left.size,1)
    
right= right.reshape(right.size,1)
    
d_channel1,d_channel2 = np.split(d, 2,axis=1)



convolution1 = scipy.signal.convolve(d_channel1,left,mode='same')

convolution2 = scipy.signal.convolve(d_channel2,right,mode='same')


transfered = np.concatenate((convolution1,convolution2),axis=1)
#sf.write(filename,transfered,sr1)

#print(reverseL)
print(left)
HRTF.close()

[[ 1.78058350e+00+0.00000000e+00j]
 [-1.81379401e+00+6.93889390e-17j]
 [ 1.49904094e+00+0.00000000e+00j]
 [-1.63268529e+00+1.40512602e-16j]
 [ 1.39091953e+00+0.00000000e+00j]
 [-1.61149139e+00+1.05492871e-16j]
 [ 1.43036118e+00+0.00000000e+00j]
 [-1.50841120e+00+5.33427469e-17j]
 [ 1.19982211e+00-9.71445147e-17j]
 [-1.28747432e+00+2.05292633e-16j]
 [ 1.02639694e+00-2.77555756e-17j]
 [-1.18385525e+00+2.07624716e-16j]
 [ 9.79416927e-01-8.67361738e-17j]
 [-1.19435656e+00+1.98679000e-16j]
 [ 9.01097167e-01-1.09070739e-16j]
 [-1.03241396e+00+2.21610924e-16j]
 [ 7.73317217e-01+0.00000000e+00j]
 [-9.54189348e-01+2.42861287e-17j]
 [ 7.26797719e-01+0.00000000e+00j]
 [-9.55578974e-01+1.23165367e-16j]
 [ 7.66937007e-01+0.00000000e+00j]
 [-9.30990373e-01+6.21247845e-17j]
 [ 6.62878803e-01+0.00000000e+00j]
 [-8.77084501e-01+8.80372164e-17j]
 [ 6.23793754e-01-2.77555756e-17j]
 [-8.31129701e-01+9.22379757e-17j]
 [ 6.26055052e-01-5.55111512e-17j]
 [-8.59994168e-01+2.84494650e-16j]
 [ 5.83096078e-01-6.

In [2]:
print( 'hpcf stands for "headphone compensation function" and HRTF stands for "head-related transfer function"')

hpcf stands for "headphone compensation function" and HRTF stands for "head-related transfer function"


In [13]:
def hpcf(pathC,pathA,filename):
    '''
    This function compensates the Frequency Response of the headset by convolving the Frequency Response compensator 
    with the audio. 
    
    The Frequency Responese of the headphone is flattened and thus is samely sensitive to all frequencies.
    
    Inputs
    ======
    The path of the Frequency Response Compensator, path C.
    
    The path of the audio file, path A.
    
    The path to write target file, filename.
    
    outputs
    ======
    
    Write the filtered audio files to the assigned location.
    
    '''
    data, sr = sf.read(pathC)
    
    data= data.reshape(data.size,1)
    
    d, sr1 = sf.read(pathA)
    
    d_channel1,d_channel2 = np.split(d, 2,axis=1)

    convolution1 = scipy.signal.convolve(d_channel1,data,mode='same')

    convolution2 = scipy.signal.convolve(d_channel2,data,mode='same')
    
    compensated = np.concatenate((convolution1,convolution2),axis=1)
    
    sf.write(filename,compensated,sr1)

    

In [12]:
pathC=f'/Users/eeeds/Desktop/yg/hpcf/HpCF_Beyerdynamic_Custom_One_Pro_E.wav'
path1=f'E://yonggang/corrected/halves/audio0001firsthalf.wav'
fhs=f'E://yonggang/hpcfE/Eaudio0001firsthalf.wav'
hpcf(pathC,path1,fhs)

[[ 2.81811376e-03]
 [ 7.57671331e-03]
 [ 7.36401408e-03]
 ...
 [-9.49770765e-05]
 [-9.42202096e-05]
 [-9.34441850e-05]]


In [33]:
pathC=f'/Users/eeeds/Desktop/yg/hpcf/HpCF_Beyerdynamic_Custom_One_Pro_E.wav'

for i in range (1,134):
    path1=f'E://yonggang/corrected/halves/audio{i:04d}firsthalf.wav'       
    
    fhs=f'E://yonggang/hpcfE/Eaudio{i:04d}firsthalf.wav'
    
    hpcf(pathC,path1,fhs)
    
    path2=f'E://yonggang/corrected/halves/audio{i:04d}secondhalf.wav'      
    
    shs=f'E://yonggang/hpcfE/Eaudio{i:04d}secondhalf.wav'
    
    hpcf(pathC,path2,shs)
            
    for j in range (1,3):
        path3 = f'E://yonggang/corrected/random/audio{i:04d}random{j:04d}.wav'      
        
        rans=f'E://yonggang/hpcfE/Eaudio{i:04d}random{j:04d}.wav'
    
        hpcf(pathC,path3,rans)

In [50]:
path = '/Users/eeeds/Desktop/yg/mit_kemar_normal_pinna.sofa'
    
HRTF = sofa.Database.open(path)

#print(np.round(HRTF.Source.Position.get_values(indices={"M":slice(260,297)}, system="spherical", angle_unit="degree"),2))

print(np.round(HRTF.Source.Position.get_values(indices={"M":260}, system="spherical", angle_unit="degree"),2))

print(np.round(HRTF.Source.Position.get_values(indices={"M":296}, system="spherical", angle_unit="degree"),2))

print(HRTF.Data.SamplingRate.get_values(indices={"M":260}))

[0.  0.  1.4]
[180.    0.    1.4]
44100.0


In [32]:
def hrtf(pathC,pathA,filename,left_mea,right_mea,emitter,receiver):
    
    #exact IR of the atificial head from .sofa file 
    HRTF = sofa.Database.open(pathC)
 
    leftIR = HRTF.Data.IR.get_values(indices={'M':left_mea,'R':receiver,'E':emitter})

    leftIR=leftIR.reshape(leftIR.size,)

    rightIR= HRTF.Data.IR.get_values(indices={'M':right_mea,'R':receiver,'E':emitter})

    rightIR=rightIR.reshape(rightIR.size,)
    

    #1-D array fourier transform.
    leftFT = scipy.fft.fft(leftIR,axis=-1) 
    
    rightFT = scipy.fft.fft(rightIR,axis=-1) 

    #inverse transform to get the time domain transfer function.
    left = scipy.fft.ifft(1/leftFT,axis=-1)
    
    right = scipy.fft.ifft(1/rightFT,axis=-1)

    #retrieve audio data and split to 2 channels
    d, sr1 = sf.read(pathA)
    
    d_channel1,d_channel2 = np.split(d, 2,axis=1)

    left= left.reshape(left.size,1)
    
    right= right.reshape(right.size,1)
    
    convolution1 = scipy.signal.convolve(d_channel1,left,mode='same')

    convolution2 = scipy.signal.convolve(d_channel2,right,mode='same')
    
    transfered = np.concatenate((convolution1,convolution2),axis=1)
    
    sf.write(filename,transfered,sr1)
    
    HRTF.close()

In [58]:

pathC = '/Users/eeeds/Desktop/yg/mit_kemar_normal_pinna.sofa'

for i in range (1,134):
    
    path1=f'E://yonggang/hpcfE/Eaudio{i:04d}firsthalf.wav'   
    
    fhname=f'E://yonggang/hrtf/HRTFaudio{i:04d}firsthalf.wav'
    
    hrtf(pathC,path1,fhname,left_mea=260,right_mea=296,emitter=0,receiver=0)
    
    path2=f'E://yonggang/hpcfE/Eaudio{i:04d}secondhalf.wav'   
    
    shname=f'E://yonggang/hrtf/HRTFaudio{i:04d}secondhalf.wav'
    
    hrtf(pathC,path2,shname,left_mea=260,right_mea=296,emitter=0,receiver=0)
            
    for j in range (1,3):
        path3 = f'E://yonggang/hpcfE/Eaudio{i:04d}random{j:04d}.wav'   
        
        rans=f'E://yonggang/hrtf/HRTFaudio{i:04d}random{j:04d}.wav'
    
        hrtf(pathC,path2,rans,left_mea=260,right_mea=296,emitter=0,receiver=0)


C:\Users\eeeds\anaconda3\envs\tf2yonggang\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\eeeds\anaconda3\envs\tf2yonggang\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
C:\Users\eeeds\anaconda3\envs\tf2yonggang\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\eeeds\anaconda3\envs\tf2yonggang\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


In [35]:
def calibration (D,path1,cn):
    '''
    This function calibrates an assigned audio file to a fixed dbFS value D.
    
    Inputs
    ======
    The desired value of dbFS, D (float).
    
    The path of the audio file, path1.
    
    The path to write the calibrated file, cn.
    
    outputs
    ======
    
    Write the calibrated audio files to the assigned location.
    
    '''
    
    data, sr = sf.read(path1)
    
    dbFS = 10 * math.log(np.mean(np.square(data)),10)
    
    X= (D - dbFS) / 20
    
    
    k = pow(10,X)       # 10lg(square(pi)) + 20lgk = D   ----> 
                        # 20lgk = D - 10lg(square(pi))   ---->
                        #  k = pow(10,((D-dbFS)/20))  
    print(k)
     
    calibrated = k * data
    
    sf.write(cn,calibrated,sr)
    

In [59]:
#calibrate all
for i in range (1,134):
    
    path1=f'E://yonggang/hrtf/HRTFaudio{i:04d}firsthalf.wav'   
    
    fhname=f'E://yonggang/cal_hrtf/calaudio{i:04d}firsthalf.wav'
    
    calibration(-37,path1,fhname)
    
    path2=f'E://yonggang/hrtf/HRTFaudio{i:04d}secondhalf.wav'   
    
    shname=f'E://yonggang/cal_hrtf/calaudio{i:04d}secondhalf.wav'
    
    calibration(-37,path2,shname)
            
    for j in range (1,3):
        path3 = f'E://yonggang/hrtf/HRTFaudio{i:04d}random{j:04d}.wav'   
        
        rans=f'E://yonggang/cal_hrtf/calaudio{i:04d}random{j:04d}.wav'
    
        calibration(-37,path3,rans)

0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754


0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754
0.01412537544622754


In [20]:
#Randomly calibrate first halves and second halves.
for j in range(1,6):
    
    i=random.randint(1,51)
    
    name_list=['firsthalf','secondhalf']
    
    name=random.choice(name_list)
    
    pathA = 'E://yonggang/hpcfE/Eaudio'+f'{i:04d}{name}.wav'
    
    cn    = f'E://yonggang/calib_hpcfE/calaudio{i:04d}{name}.wav'
    
    calibration(-37,pathA,cn)

0.6919220525492363
2.208959401679008
0.30507500670007276
1.0003237314390723
1.2320304380646043


In [24]:
#randomly calibrate random pieces
for j in range(1,6):
    
    i=random.randint(1,51)
    
    j=random.randint(1,2)
    
    pathA = 'E://yonggang/hpcfE/Eaudio'+f'{i:04d}random{j:04d}.wav'
    
    cn    = f'E://yonggang/calib_hpcfE/calaudio{i:04d}random{j:04d}.wav'
     
    calibration(-37,pathA,cn)

0.40553287888941364
0.8001320696837062
1.5717117249440051
0.8048403436817639
1.2500460667702324


In [67]:
dbfs('E://yonggang/hrtf/HRTFaudio0017firsthalf.wav')

0.0

In [60]:
def dbfs(path):
    '''
    This function caculates the Decibels relative to the Full Scale (dbFS) of an audio file.
    
    Inputs
    ======
    
    The path of the audio file, path A.
    
    outputs
    ======
    
    Return the dbFS value of input file.
    
    '''
    
    #path=f'E://yonggang/exp/{name}.wav'
    
    d1, sr1 = sf.read(path)
    
    dbFS = 10*math.log(np.mean(np.square(d1)),10)
    
    return dbFS

In [16]:
pathC=f'/Users/eeeds/Desktop/yg/hpcf/HpCF_Beyerdynamic_Custom_One_Pro_E.wav'

pathA=f'E://yonggang/corrected/halves/audio0001firsthalf.wav' 

data, sr = sf.read(pathC)
    
data= data.reshape(data.size,1)
    
d, sr1 = sf.read(pathA)
    
d_channel1,d_channel2 = np.split(d, 2,axis=1)

convolution1 = scipy.signal.convolve(d_channel1,data,mode='same')

convolution2 = scipy.signal.convolve(d_channel2,data,mode='same')
    
compensated = np.concatenate((convolution1,convolution2),axis=1)
print(compensated.shape)
#print(convolution2)
#print(convolution1)
# print(d_channel1)
x = np.random.rand(3,1)
y=np.random.rand(3,1)
z=np.concatenate((x,y),axis=1)
print(data,
      y,
      z)

(1323000, 2)
[[ 9.99999881e-01]
 [-2.85832524e-01]
 [-1.33473516e-01]
 ...
 [-1.19209290e-07]
 [-1.19209290e-07]
 [-1.19209290e-07]] [[0.82571744]
 [0.90061538]
 [0.29675987]] [[0.54622841 0.82571744]
 [0.2919512  0.90061538]
 [0.02580662 0.29675987]]
